In [1]:
from Loader import fileloader as fl
from Loader import datavalidator as dv
from Loader import datahandledatabase as dhdb

month = '201809'
testpath = 'D:/Datos de Usuario/cleon/Documents/Mercado Empresas/Data Fuente Comisiones/test/'
defaultpath = 'D:/Datos de Usuario/cleon/Documents/Mercado Empresas/Data Fuente Comisiones/xlsx/'


In [2]:
inifile = fl.ReadIniFile('C:/Anaconda3/MeScripts/Comisiones4/Config/myconfig.ini')
parser = inifile.readFile()

loader = fl.LoadFileProcess(month)
loader.setParser(parser)
loader.setDefaultPath(defaultpath)

In [3]:
# Importando la información
# Validar en los equipos quincenales la jerarquia de ventas
# Copiar los reportes comisionales del periodo : Grandes Cuentas y Pymes

# Inar y eliminando las migraciones

#"""
inarbruto = loader.loadFile('Inar_bruto')
jerarquia = loader.loadFile('Jerarquia')
comisionantes = loader.loadFile('Comisionantes_voz')
#"""

inidb = fl.ReadIniFile('C:/Anaconda3/MeScripts/Comisiones4/Config/mydbconfig.ini')
dbparser = inidb.readFile()

dbmanager = dhdb.DbDataProcess(month)
dbmanager.setParser(dbparser)
tblempleados = dbmanager.loadData('tblEmpleados')

El tamaño de Inar_bruto es 215177 registros
Hoja Importada: JERARQUIA
El tamaño de Jerarquia es 30725 registros
El tamaño de Comisionantes_voz es 314 registros
Los registros de la tabla tblEmpleados es 994 registros  


In [4]:
# Armando las validaciones

valinarfl = {'inarbruto' : inarbruto, 'jerarquia' : jerarquia, 'comisionantes' : comisionantes, 'tblempleados' : tblempleados}
valinarobj = dv.ValidateInar(valinarfl)
inarvalidation = valinarobj.validation()


valmultipledatafl = {'jerarquia' : jerarquia, 'comisionantes' : comisionantes, 'tblempleados' : tblempleados}
valmultipledataobj = dv.ValidateMultipleData(valmultipledatafl)
multipledatavalidation = valmultipledataobj.validation()

#Exportando validaciones

xlsxfile1 = testpath + month + '_Validaciones en INAR.xlsx'
exportparams = {'xlsxfile' : xlsxfile1, 'dataframe' : inarvalidation}
valinarobj.exportValidation(exportparams)

#"""
xlsxfile2 = testpath + month + '_Otras Validaciones.xlsx'
exportparams = {'xlsxfile' : xlsxfile2, 'dataframe' : multipledatavalidation}
valmultipledataobj.exportValidation(exportparams)
#"""

C:\Anaconda3\MeScripts\Comisiones4\Loader\datavalidator.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  blanksdoc['OBSERVACION'] = 'blanks en documento'
C:\Anaconda3\MeScripts\Comisiones4\Loader\datavalidator.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  blanksvendedor['OBSERVACION'] = 'blanks en vendedor o zona'


Archivo exportado D:/Datos de Usuario/cleon/Documents/Mercado Empresas/Data Fuente Comisiones/test/201809_Validaciones en INAR.xlsx con 319 registros
Archivo exportado D:/Datos de Usuario/cleon/Documents/Mercado Empresas/Data Fuente Comisiones/test/201809_Otras Validaciones.xlsx con 45 registros


C:\Anaconda3\MeScripts\Comisiones4\Loader\datavalidator.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  conjerarquia['CONCOLSJER'] = conjerarquia.apply(lambda x : '|'.join(x), axis = 1)


In [6]:
# Detectando los registros de Tienda que no tienen Orden
str_tienda = 'Tienda'
lst_estados = ['NEWS','ADDS','FCHURN','DEOF','REAC']

mask0 = inarbruto['ESTADO'].isin(lst_estados)
mask1 = inarbruto['NRO_ORDEN'].isnull()
inar_f1 = inarbruto[mask0 & mask1]
mask2 = inar_f1['VENDEDOR'].notnull()
inar_f2 = inar_f1[mask2]
mask3 = inar_f2['VENDEDOR'].str.contains(str_tienda)
inar_f3 = inar_f2[mask3]

#mask2 = inarbruto['VENDEDOR'].str.contains(str_tienda)
#inar_filtrado = inarbruto[mask2]



In [7]:
inar_f2.to_csv(testpath + 'inar_sin_orden.csv')